In [2]:
import numpy as np
import os
import torch
print(torch.__version__)
import torch.nn.functional as F
import torch.nn as nn

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from functools import partial
from torch.utils.data import TensorDataset, DataLoader

2.0.1


In [3]:
import pandas as pd

In [4]:
all_county = pd.read_csv("income-dsci-temp-data.csv")
Accomack = all_county.query("County=='Accomack'")

In [19]:
Accomack

,County,Year,Income,DSCI,Precip,Temp
0,Accomack,2008,32179,97.962264,44.92,58.5
1,Accomack,2009,32962,29.538462,55.34,57.8
2,Accomack,2010,33744,53.788462,38.83,58.3
3,Accomack,2011,33794,70.000000,40.85,59.5
4,Accomack,2012,35201,80.788462,49.68,60.4
5,Accomack,2013,35412,3.698113,47.18,57.5
6,Accomack,2014,36346,18.923077,41.92,57.1
7,Accomack,2015,37523,12.711538,46.71,58.4
8,Accomack,2016,39924,6.480769,56.11,59.3
9,Accomack,2017,43521,7.884615,47.94,59.6


In [5]:
crop = pd.read_csv("crop_data.csv")
Acc_crop = crop.query("County=='Accomack'")
crop_replace_map = { 'Corn': 'Crops', 'Other_Hay/Non_Alfalfa': 'Crops', 'Soybeans': 'Crops', 'Alfalfa':'Crops', 'Sorghum':'Crops', 'Dbl_Crop_WinWht/Soybeans':'Crops', 'Winter_Wheat':'Crops', 'Dbl_Crop_WinWht/Corn':'Crops', 'Millet':'Crops', 'Dbl_Crop_Barley/Soybeans':'Crops', 'Rye':'Crops', 'Barley':'Crops', 'Dbl_Crop_Soybeans/Oats ':'Crops', 'Dbl_Crop_Barley/Corn':'Crops', 'Other_Crops':'Crops', 'Dbl_Crop_WinWht/Sorghum':'Crops', 'Oats':'Crops', 'Cotton':'Crops', 'Apples':'Crops', 'Pumpkins':'Crops', 'Tobacco':'Crops', 'Sunflower':'Crops', 'Christmas_Trees':'Crops', 'Triticale':'Crops', 'Grapes':'Crops', 'Sweet_Corn':'Crops', 'Peanuts':'Crops', 'Dry_Beans':'Crops', 'Dbl_Crop_Oats/Corn':'Crops', 'Potatoes':'Crops', 'Peaches':'Crops', 'Misc_Vegs_&_Fruits':'Crops', 'Dbl_Crop_Barley/Sorghum':'Crops', 'Sweet_Potatoes':'Crops', 'Squash':'Crops', 'Tomatoes':'Crops', 'Strawberries':'Crops', 'Broccoli':'Crops', 'Watermelons':'Crops', 'Other_Tree_Crops':'Crops', 'Canola':'Crops', 'Cantaloupes':'Crops', 'Cabbage':'Crops', 'Vetch':'Crops', 'Walnuts':'Crops', 'Rape_Seed':'Crops', 'Dbl_Crop_WinWht/Cotton':'Crops', 'Dbl_Crop_Corn/Soybeans':'Crops', 'Cherries':'Crops', 'Buckwheat':'Crops', 'Dbl_Crop_Soybeans/Cotton':'Crops', 'Switchgrass':'Crops', 'Cucumbers':'Crops', 'Cranberries':'Crops', 'Pop_or_Orn_Corn':'Crops', 'Radishes':'Crops', 'Greens':'Crops', 'Pears':'Crops', 'Peppers':'Crops', 'Pecans':'Crops', 'Peas':'Crops', 'Durum_Wheat':'Crops', 'Gourds':'Crops', 'Spring_Wheat':'Crops', 'Speltz':'Crops', 'Turnips':'Crops', 'Grassland/Pasture': 'Grassland', 'Sod/Grass_Seed': 'Grassland', 'Switchgrass':'Grassland', 'Mixed_Forest': 'Forest', 'Evergreen_Forest': 'Forest', 'Deciduous_Forest': 'Forest', 'Barren': 'Barren', 'Fallow/Idle_Cropland': 'Barren', 'Developed/High_Intensity': 'Developed', 'Developed/Med_Intensity': 'Developed', 'Developed/Low_Intensity': 'Developed', 'Developed/Open_Space': 'Developed', 'Open_Water': 'Open_Water', 'Shrubland': 'Shrubland', 'Woody_Wetlands':'Wetlands', 'Herbaceous_Wetlands':'Wetlands', 'Aquaculture':'Aquaculture' } 

In [6]:
Acc_crop["Crop"] = Acc_crop["Crop"].map(crop_replace_map)
Acc_crop = Acc_crop[["Percentage", "Year", "Crop"]]
Acc = Acc_crop.groupby(["Crop", "Year"]).sum().reset_index()

C:\Users\daven\AppData\Local\Temp\ipykernel_15484\4274211684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Acc_crop["Crop"] = Acc_crop["Crop"].map(crop_replace_map)


In [7]:
merged = pd.merge(Acc, Accomack, on="Year", how="left")

In [8]:
merged

,Crop,Year,Percentage,County,Income,DSCI,Precip,Temp
0,Aquaculture,2016,0.000018,Accomack,39924,6.480769,56.11,59.3
1,Aquaculture,2017,0.000024,Accomack,43521,7.884615,47.94,59.6
2,Aquaculture,2018,0.000017,Accomack,44919,31.538462,51.95,59.1
3,Aquaculture,2019,0.000017,Accomack,47778,7.622642,45.89,59.8
4,Aquaculture,2020,0.000073,Accomack,50924,14.442308,55.07,60.1
...,...,...,...,...,...,...,...,...
121,Wetlands,2018,0.086843,Accomack,44919,31.538462,51.95,59.1
122,Wetlands,2019,0.120213,Accomack,47778,7.622642,45.89,59.8
123,Wetlands,2020,0.120692,Accomack,50924,14.442308,55.07,60.1
124,Wetlands,2021,0.119864,Accomack,56453,43.557692,39.43,59.5


In [9]:
merged.sort_values(by="Year")

,Crop,Year,Percentage,County,Income,DSCI,Precip,Temp
37,Developed,2008,0.010447,Accomack,32179,97.962264,44.92,58.5
111,Wetlands,2008,0.084279,Accomack,32179,97.962264,44.92,58.5
82,Open_Water,2008,0.008113,Accomack,32179,97.962264,44.92,58.5
22,Crops,2008,0.055046,Accomack,32179,97.962264,44.92,58.5
7,Barren,2008,0.002060,Accomack,32179,97.962264,44.92,58.5
...,...,...,...,...,...,...,...,...
51,Developed,2022,0.019349,Accomack,54153,100.403846,40.56,58.9
96,Open_Water,2022,0.012826,Accomack,54153,100.403846,40.56,58.9
66,Forest,2022,0.013735,Accomack,54153,100.403846,40.56,58.9
110,Shrubland,2022,0.000231,Accomack,54153,100.403846,40.56,58.9


In [ ]:
#input features: year, income, DSCI, precip, and temp
#output feature: crop percentage

In [19]:
merged.query("Year == 2009")

,Crop,Year,Percentage,County,Income,DSCI,Precip,Temp
8,Barren,2009,0.003260,Accomack,32962,29.538462,55.34,57.8
23,Crops,2009,0.052992,Accomack,32962,29.538462,55.34,57.8
38,Developed,2009,0.010476,Accomack,32962,29.538462,55.34,57.8
53,Forest,2009,0.004375,Accomack,32962,29.538462,55.34,57.8
68,Grassland,2009,0.001199,Accomack,32962,29.538462,55.34,57.8
83,Open_Water,2009,0.008442,Accomack,32962,29.538462,55.34,57.8
98,Shrubland,2009,0.001911,Accomack,32962,29.538462,55.34,57.8
112,Wetlands,2009,0.085065,Accomack,32962,29.538462,55.34,57.8


In [10]:
X = merged[["Year","Income","DSCI","Precip","Temp"]].to_numpy()

In [11]:
y = merged["Percentage"].to_numpy()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=49)

In [40]:
#training data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

#testing data
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [13]:
import tensorflow as tf

In [14]:
from keras.models import Sequential
from keras.layers import Dense

In [51]:
y_train_tensor.size()

torch.Size([100])

In [49]:
X_train_tensor.size()

torch.Size([100, 5])

In [15]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=5)) #5 input variables; 128 nodes in hidden layer
model.add(Dense(9, activation='softmax')) #output 9 types of landuse
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
hist = model.fit(X_train, y_train, epochs=40, batch_size=10, validation_split=0.2) 



Epoch 1/40


8/8 [==============================] - 1s 52ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 2/40
8/8 [==============================] - 0s 9ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 3/40
8/8 [==============================] - 0s 10ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 4/40
8/8 [==============================] - 0s 10ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 5/40
8/8 [==============================] - 0s 9ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 6/40
8/8 [==============================] - 0s 9ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 7/40
8/8 [==============================] - 0s 9ms/step - loss: 0.1080 - mse: 0.1080 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 8/40
8/8 [==============================] - 0s 9ms/step - loss: 0.1080 - mse: 0.1080 - val_

In [20]:
loss, mae = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 161ms/step - loss: 0.1077 - mse: 0.1077
